<a href="https://colab.research.google.com/github/piyushgupta198/Medical-Report-Assistant/blob/main/Medical_Report_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -qU huggingface_hub langchain_huggingface langchain-community faiss-cpu pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [3]:
%pip install -qU langchain_community

In [4]:
from google.colab import userdata
sec_key = userdata.get('HF_TOKEN')

# Load PDF & Parsing

In [6]:
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader("/content/475165825_SL_2024-12-14T173424.48_report (1).pdf")

In [7]:
docs = loader.load()

In [8]:
report_text = " ".join(chunk.page_content for chunk in docs)

In [9]:
# print(report_text)

In [10]:
cleaned_text = report_text.replace('|', '').strip()
cleaned_report_text = cleaned_text.replace('-', '').strip()

# Text Splitting

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
chunks = splitter.create_documents([cleaned_report_text])

In [12]:
len(chunks)

65

## Embedding Generation and Storing in Vector Store

In [13]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings
em_model = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
embedding_model = HuggingFaceEndpointEmbeddings(
    model=em_model,
    huggingfacehub_api_token=sec_key,
)

In [14]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(chunks, embedding_model)

In [15]:
# vector_store.index_to_docstore_id

In [16]:
print(vector_store.get_by_ids(['925fcf2f-4f3b-4add-9f91-bce1a00f8d77']))

[]


# Retrieval


In [47]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [18]:
retriever.invoke('CBC')

[Document(id='f8afc6ef-0af9-4c42-b0b6-a6ed74d69b6d', metadata={}, page_content='E3, OLD NO2/29, MAIN BABARPUR ROAD, Plot no. 33, Defence Enclave, Vikas Marg,\nDELHI Preet Vihar, New Delhi110092\nNORTH EAST DELHI\nTest Report\nTest Name Results Units Bio. Ref. Interval\nCOMPLETE BLOOD COUNT;CBC\nHemoglobin 9.90 g/dL 12.00  15.00\n(Photometry)'),
 Document(id='6b1bf2c7-17f2-4efa-b587-055972f12022', metadata={}, page_content='E3, OLD NO2/29, MAIN BABARPUR ROAD, Plot no. 33, Defence Enclave, Vikas Marg,\nDELHI Preet Vihar, New Delhi110092\nNORTH EAST DELHI\nTest Report\nTest Name Results Units Bio. Ref. Interval\nKIDNEY PANEL; KFT,SERUM\nCreatinine 0.56 mg/dL 0.55  1.02\n(Modified Jaffe,Kinetic)'),
 Document(id='14209cf7-f5b8-41f6-b263-a568bc1963f2', metadata={}, page_content='Centromere  AC3  CENPA/B  Limited cutaneous SSc, PBC \n\n Discrete nuclear  AC6,7   \n dots    \n\n Multiple nuclear  AC6  Sp100, PML  PBC, SARD, PM/DM \n dots   proteins,  \n   MJ/NXP2  \n\n Few nuclear dots  AC7  P

# Augmentation

In [19]:
import os
os.environ["HF_TOKEN"] = sec_key

In [20]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [39]:
from langchain_huggingface import HuggingFaceEndpoint

In [22]:
from huggingface_hub import login
login(new_session=False)

In [42]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Devstral-Small-2507",
    provider="featherless-ai",
    temperature=0.1,
    do_sample=False,
    huggingfacehub_api_token=sec_key
)

In [44]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(
    template="""
    You are a clinical assistant AI trained to interpret medical blood test reports.
    Given the following lab results and information, answer the user's question clearly, based only on the medical data provided. Use clinical reasoning and reference values where appropriate.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [48]:
question = "lupus"
retrieved_docs = retriever.invoke(question)

In [49]:
retrieved_docs

[Document(id='728068fa-815c-43bd-bc55-8ae8a17407cd', metadata={}, page_content='Mitotic chromosome  AC28  Modified histone  Rare in Discoid lupus erythematous, \n coat   H3, MCA1  Chronic lymphocytic leukemia, \n    Sjogren’s syndrome, and Polymyalgia \n    rheumatica'),
 Document(id='0102e019-c452-439a-8d28-95c3f4c3bb4c', metadata={}, page_content='Homogenous  AC1  dsDNA*, 4nu7cle5os1om6es,5 8 S2LE5, D*ruginduced lupus, \n   histones  Juvenile idiopathic arthritis \nPage 6 of 9\n .\nName : Ms. YASHIKA\nLab No. : 475165825 Age : 23 Years\nRef By : DR RAHUL BAJAJ Gender : Female\nCollected : 12/12/2024 11:47:00AM Reported : 14/12/2024 5:30:16PM'),
 Document(id='964aab5a-0453-4714-a932-637dcd13c146', metadata={}, page_content='Segmental  AC17  Alphaactin,  Myasthenia gravis, Crohn’s disease, \n   vinuculin,  Ulcerative colitis \n   tropomyosin  \n\n Speckled  AC18,   \n  19,20   \n\n Discrete dots/  AC18  SGW182, Su/Ago2,  PBC, SARD, neurological and \n GW body like    autoimmune conditi

In [50]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
print(context_text)
final_prompt = prompt.invoke({"context": context_text, "question": question})

final_prompt

Mitotic chromosome  AC28  Modified histone  Rare in Discoid lupus erythematous, 
 coat   H3, MCA1  Chronic lymphocytic leukemia, 
    Sjogren’s syndrome, and Polymyalgia 
    rheumatica

Homogenous  AC1  dsDNA*, 4nu7cle5os1om6es,5 8 S2LE5, D*ruginduced lupus, 
   histones  Juvenile idiopathic arthritis 
Page 6 of 9
 .
Name : Ms. YASHIKA
Lab No. : 475165825 Age : 23 Years
Ref By : DR RAHUL BAJAJ Gender : Female
Collected : 12/12/2024 11:47:00AM Reported : 14/12/2024 5:30:16PM

Segmental  AC17  Alphaactin,  Myasthenia gravis, Crohn’s disease, 
   vinuculin,  Ulcerative colitis 
   tropomyosin  

 Speckled  AC18,   
  19,20   

 Discrete dots/  AC18  SGW182, Su/Ago2,  PBC, SARD, neurological and 
 GW body like    autoimmune conditions

Nucleolar punctate  AC10  RNA polymerase 1,  SSc, Sjogren’s syndrome 
   hUBF/NOR90  


 Nuclear envelope  AC11,   
  12   

 Smooth nuclear  AC11  Lamin A,B,C or  SLE, Sjogren’s syndrome, 
 envelope   lamin associated  Seronegative arthritis 
   proteins



StringPromptValue(text="\n    You are a clinical assistant AI trained to interpret medical blood test reports.\n    Given the following lab results and information, answer the user's question clearly, based only on the medical data provided. Use clinical reasoning and reference values where appropriate.\n\n      Mitotic chromosome  AC28  Modified histone  Rare in Discoid lupus erythematous, \n coat   H3, MCA1  Chronic lymphocytic leukemia, \n    Sjogren’s syndrome, and Polymyalgia \n    rheumatica\n\nHomogenous  AC1  dsDNA*, 4nu7cle5os1om6es,5 8 S2LE5, D*ruginduced lupus, \n   histones  Juvenile idiopathic arthritis \nPage 6 of 9\n .\nName : Ms. YASHIKA\nLab No. : 475165825 Age : 23 Years\nRef By : DR RAHUL BAJAJ Gender : Female\nCollected : 12/12/2024 11:47:00AM Reported : 14/12/2024 5:30:16PM\n\nSegmental  AC17  Alphaactin,  Myasthenia gravis, Crohn’s disease, \n   vinuculin,  Ulcerative colitis \n   tropomyosin  \n\n Speckled  AC18,   \n  19,20   \n\n Discrete dots/  AC18  SGW182, S

## Generation

In [51]:
answer = llm.invoke(final_prompt)
print(answer)

 Answer: Based on the provided lab results, there are several findings that could be associated with lupus. The presence of homogenous AC1 (dsDNA, histones) and dense fine speckled AC19 (PL7, PL12, ribosomal P proteins) patterns are particularly notable. These patterns are often seen in systemic lupus erythematosus (SLE) and other autoimmune conditions. Additionally, the nucleolar punctate AC10 pattern, which is associated with SSc and Sjogren’s syndrome, could also be relevant. However, a definitive diagnosis of lupus would require further clinical evaluation and consideration of other diagnostic criteria.


# Build Chain

In [52]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [53]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [54]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [55]:
parser = StrOutputParser()
main_chain = parallel_chain | prompt | llm | parser

In [57]:
result = main_chain.invoke('give insights from the report')

In [58]:
print(result)

 Answer: The lab report for Ms. Yashika, a 23-year-old female, includes several tests related to autoimmune conditions. Here are the key insights:

1. **AC1 (dsDNA, histones, nucleosomes)**: Elevated levels of these antibodies are associated with systemic lupus erythematosus (SLE) and drug-induced lupus. The presence of these antibodies suggests a possible autoimmune disorder.

2. **AC24 (Pericentrin, ninein, Cep250, Cep110)**: These antibodies are rare and can be associated with conditions like SSc (systemic sclerosis), Raynaud’s phenomenon, and infections (viral and mycoplasma).

3. **AC25 (HsEg5)**: Elevated levels of this antibody are rare and can be associated with Sjogren’s syndrome, SLE, and other SARD (Systemic Autoimmune Rheumatic Diseases).

4. **AC13 (PCNA)**: Elevated levels of this antibody are associated with SLE and other conditions.

The report suggests a strong possibility of an autoimmune disorder, possibly SLE, given the presence of multiple antibodies associated wit

In [61]:
print(main_chain.invoke('what about kidney?'))

 Answer: Based on the provided lab results, the kidney function test (KFT) shows a creatinine level of 0.56 mg/dL, which is within the normal reference interval (0.55 - 1.02 mg/dL). This indicates that the kidney function is normal. Additionally, the eGFR category G1 or G2 suggests that there is no evidence of chronic kidney disease (CKD) in the absence of kidney damage. Therefore, based on the provided data, the kidney function appears to be normal.
